In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Configuración visual mejorada
plt.style.use('seaborn-v0_8')
sns.set_palette("Set2")
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.facecolor'] = 'white'

print("📊 ANÁLISIS DE OCUPACIÓN EDUCATIVA - SOLICITUDES VS CAPACIDAD")
print("="*65)


In [ ]:
# === CARGA DE DATOS ===
def load_education_data():
    """Carga solo los datasets educativos necesarios"""
    data = {}

    # Datos educativos principales
    files = {
        'education': '../downloads/normalizacion/education.csv',
        'admissions': '../downloads/normalizacion/education_admition.csv',
        'enrollment': '../downloads/normalizacion/education_enrollment.csv',
        'edu_municipality': '../downloads/normalizacion/education_municipality.csv',
        'municipality': '../downloads/normalizacion/municipality.csv'
    }

    for name, path in files.items():
        delimiter = ';' if 'education' in path and 'municipality' not in path else ','
        data[name] = pd.read_csv(path, delimiter=delimiter)
        # Convertir IDs a string para consistencia
        if 'id' in data[name].columns:
            data[name]['id'] = data[name]['id'].astype(str)
        if 'id_education' in data[name].columns:
            data[name]['id_education'] = data[name]['id_education'].astype(str)
        if 'id_municipality' in data[name].columns:
            data[name]['id_municipality'] = data[name]['id_municipality'].astype(str)

    return data

data = load_education_data()
print(f"✅ Datos educativos cargados:")
for key, df in data.items():
    print(f"   • {key}: {len(df):,} registros")


In [ ]:
# === CONFIGURACIÓN DE CICLOS EDUCATIVOS ===
def get_cycle_config():
    """Configuración de ciclos con años de permanencia"""
    return {
        'infantil_i_ciclo': {
            'name': 'Infantil I',
            'años_permanencia': 3  # 0-2 años
        },
        'infantil_ii_ciclo': {
            'name': 'Infantil II',
            'años_permanencia': 3  # 3-5 años
        },
        'primaria': {
            'name': 'Primaria',
            'años_permanencia': 6  # 6-11 años
        },
        'eso': {
            'name': 'ESO',
            'años_permanencia': 4  # 12-15 años
        }
    }

cycle_config = get_cycle_config()
print("📚 Configuración de ciclos educativos:")
for cycle, config in cycle_config.items():
    print(f"   • {config['name']}: {config['años_permanencia']} años de permanencia")


In [ ]:
# === ANÁLISIS DE OCUPACIÓN POR CENTRO EDUCATIVO ===
def analyze_center_occupancy(data, cycle_config):
    """Analiza ocupación de centros: solicitudes vs capacidad estimada"""

    target_year = "2022-2023"  # Año para comparación de ocupación actual

    # FILTRO INICIAL: Solo trabajar con los ciclos específicos
    valid_cycles = ['infantil_i_ciclo', 'infantil_ii_ciclo', 'primaria', 'eso']

    # Para admisiones, usar TODOS los años disponibles para encontrar el máximo
    admissions = data['admissions'].copy()
    # Filtrar solo los ciclos válidos
    admissions = admissions[admissions['cycle'].isin(valid_cycles)]

    # Para matrícula, usar TODOS los años disponibles para encontrar el máximo
    enrollment = data['enrollment'].copy()
    # Filtrar solo los ciclos válidos
    enrollment = enrollment[enrollment['cycle'].isin(valid_cycles)]

    print(f"🎯 Analizando ocupación actual para año: {target_year}")
    print(f"   • Ciclos incluidos: {valid_cycles}")
    print(f"   • Registros de admisiones (filtrados): {len(admissions):,}")
    print(f"   • Registros de matrícula (filtrados): {len(enrollment):,}")
    print(f"   • Años disponibles: {sorted(set(enrollment['year'].unique()) | set(admissions['year'].unique()))}")

    # FILTRO PREVIO: Solo analizar centros que tienen datos en los ciclos válidos
    centros_con_ciclos_validos = set()

    # Identificar centros que tienen datos de matrícula en ciclos válidos
    centros_enrollment = enrollment['id_education'].unique()
    centros_con_ciclos_validos.update(centros_enrollment)

    # Identificar centros que tienen datos de admisiones en ciclos válidos
    centros_admissions = admissions['id_education'].unique()
    centros_con_ciclos_validos.update(centros_admissions)

    print(f"   • Centros con datos en ciclos válidos: {len(centros_con_ciclos_validos)}")

    # Análisis por centro educativo
    center_analysis = []

    for center_id in centros_con_ciclos_validos:  # Solo analizar centros relevantes
        center_info = data['education'][data['education']['id'] == center_id]
        if center_info.empty:
            continue  # Saltar si no existe información del centro

        center_info = center_info.iloc[0]
        center_data = {
            'id_education': center_id,
            'center_name': center_info['name_short'],
            'center_type': center_info.get('description_short', 'No especificado')
        }

        # Variables para totales del centro
        total_plazas_estimadas = 0
        total_max_matriculados = 0
        total_matriculados_año_objetivo = 0
        total_solicitudes_presentadas = 0
        total_solicitudes_admitidas = 0
        ciclos_activos = 0

        # Analizar cada ciclo (solo los válidos)
        for cycle in valid_cycles:
            config = cycle_config[cycle]

            # y luego tomar el máximo

            # Obtener datos de matrícula para este ciclo en todos los años
            cycle_enrollment_all_years = enrollment[
                (enrollment['id_education'] == center_id) &
                (enrollment['cycle'] == cycle)
            ]

            # Obtener datos de admisiones para este ciclo en todos los años
            cycle_admissions_all_years = admissions[
                (admissions['id_education'] == center_id) &
                (admissions['cycle'] == cycle) &
                (admissions['type_solicitude'] == 'Admitidas')
            ]

            # Calcular plazas estimadas por año: (matriculados + admitidos) ÷ años_permanencia
            plazas_por_año = {}

            # Obtener todos los años que tienen datos
            años_matricula = set(cycle_enrollment_all_years['year'].unique())
            años_admisiones = set(cycle_admissions_all_years['year'].unique())
            todos_los_años = años_matricula | años_admisiones

            for año in todos_los_años:
                # Matriculados en este año
                matriculados_año = cycle_enrollment_all_years[
                    cycle_enrollment_all_years['year'] == año
                ]['total'].sum()

                # Admitidos en este año
                admitidos_año = cycle_admissions_all_years[
                    cycle_admissions_all_years['year'] == año
                ]['total'].sum()

                # Plazas estimadas para este año
                if matriculados_año > 0 or admitidos_año > 0:
                    plazas_año = matriculados_año / config['años_permanencia']
                    plazas_por_año[año] = {
                        'matriculados': matriculados_año,
                        'admitidos': admitidos_año,
                        'total_estudiantes': matriculados_año,
                        'plazas_estimadas': plazas_año
                    }

            # Tomar el MÁXIMO de plazas estimadas entre todos los años
            if plazas_por_año:
                año_max_plazas = max(plazas_por_año.keys(), key=lambda x: plazas_por_año[x]['plazas_estimadas'])
                max_plazas_data = plazas_por_año[año_max_plazas]

                plazas_estimadas = max_plazas_data['plazas_estimadas']
                max_matriculados = max_plazas_data['matriculados']
                max_admitidos = max_plazas_data['admitidos']
                max_total_estudiantes = max_plazas_data['total_estudiantes']
                año_max = año_max_plazas
            else:
                plazas_estimadas = 0
                max_matriculados = 0
                max_admitidos = 0
                max_total_estudiantes = 0
                año_max = 'N/A'

            # Datos del año objetivo para comparación de ocupación actual
            cycle_enrollment_target = enrollment[
                (enrollment['id_education'] == center_id) &
                (enrollment['cycle'] == cycle) &
                (enrollment['year'] == target_year)
            ]
            matriculados_año_objetivo = cycle_enrollment_target['total'].sum()

            # Solicitudes presentadas del año objetivo
            cycle_admissions_target = admissions[
                (admissions['id_education'] == center_id) &
                (admissions['cycle'] == cycle) &
                (admissions['year'] == target_year)
            ]

            solicitudes_presentadas = cycle_admissions_target[
                cycle_admissions_target['type_solicitude'] == 'Presentadas'
            ]['total'].sum()
            solicitudes_admitidas = cycle_admissions_target[
                cycle_admissions_target['type_solicitude'] == 'Admitidas'
            ]['total'].sum()

            if cycle == 'primaria':
                # Verificar si el centro también ofrece infantil II (usar máximo también)
                infantil_ii_enrollment_all = enrollment[
                    (enrollment['id_education'] == center_id) &
                    (enrollment['cycle'] == 'infantil_ii_ciclo')
                ]

                infantil_ii_admissions_all = admissions[
                    (admissions['id_education'] == center_id) &
                    (admissions['cycle'] == 'infantil_ii_ciclo') &
                    (admissions['type_solicitude'] == 'Admitidas')
                ]

                if not infantil_ii_enrollment_all.empty or not infantil_ii_admissions_all.empty:
                    # Calcular el máximo de infantil II usando la misma lógica
                    plazas_infantil_por_año = {}
                    años_inf_matricula = set(infantil_ii_enrollment_all['year'].unique())
                    años_inf_admisiones = set(infantil_ii_admissions_all['year'].unique())
                    todos_años_inf = años_inf_matricula | años_inf_admisiones

                    for año in todos_años_inf:
                        mat_inf = infantil_ii_enrollment_all[
                            infantil_ii_enrollment_all['year'] == año
                        ]['total'].sum()

                        adm_inf = infantil_ii_admissions_all[
                            infantil_ii_admissions_all['year'] == año
                        ]['total'].sum()

                        if mat_inf > 0 or adm_inf > 0:
                            plazas_inf_año = (mat_inf + adm_inf) / cycle_config['infantil_ii_ciclo']['años_permanencia']
                            plazas_infantil_por_año[año] = plazas_inf_año

                    if plazas_infantil_por_año:
                        max_plazas_infantil = max(plazas_infantil_por_año.values())
                        # Transiciones automáticas basadas en el máximo de plazas de infantil II
                        transiciones_automaticas = max_plazas_infantil
                    else:
                        transiciones_automaticas = 0

                    # SOLICITUDES ADMITIDAS CORREGIDAS = Admisiones + Transiciones automáticas
                    solicitudes_admitidas_corregidas = solicitudes_admitidas + transiciones_automaticas
                    center_data[f'{cycle}_transiciones_infantil'] = transiciones_automaticas
                    center_data[f'{cycle}_max_plazas_infantil_ii'] = max_plazas_infantil if plazas_infantil_por_año else 0
                else:
                    solicitudes_admitidas_corregidas = solicitudes_admitidas
                    center_data[f'{cycle}_transiciones_infantil'] = 0
                    center_data[f'{cycle}_max_plazas_infantil_ii'] = 0
            else:
                solicitudes_admitidas_corregidas = solicitudes_admitidas
                center_data[f'{cycle}_transiciones_infantil'] = 0

            # Tasa de ocupación: matriculados actuales / máximo histórico matriculados
            tasa_ocupacion_ciclo = (matriculados_año_objetivo / max_matriculados) if max_matriculados > 0 else 0

            # Ratio demanda: solicitudes presentadas / plazas estimadas
            ratio_demanda_ciclo = (solicitudes_presentadas / plazas_estimadas) if plazas_estimadas > 0 else 0

            # Ratio admisión vs capacidad: solicitudes admitidas / plazas estimadas
            ratio_admision_vs_capacidad = (solicitudes_admitidas_corregidas / plazas_estimadas) if plazas_estimadas > 0 else 0

            # Para primaria, excluir las transiciones automáticas del cálculo de eficiencia
            eficiencia_admision = (solicitudes_admitidas / solicitudes_presentadas) if solicitudes_presentadas > 0 else 0

            # Guardar métricas del ciclo
            center_data.update({
                f'{cycle}_max_matriculados': max_matriculados,
                f'{cycle}_max_admitidos': max_admitidos,
                f'{cycle}_max_total_estudiantes': max_total_estudiantes,
                f'{cycle}_año_max': año_max,
                f'{cycle}_matriculados_objetivo': matriculados_año_objetivo,
                f'{cycle}_plazas_estimadas': plazas_estimadas,
                f'{cycle}_solicitudes_presentadas': solicitudes_presentadas,
                f'{cycle}_solicitudes_admitidas': solicitudes_admitidas,
                f'{cycle}_solicitudes_admitidas_corregidas': solicitudes_admitidas_corregidas,
                f'{cycle}_tasa_ocupacion': tasa_ocupacion_ciclo,
                f'{cycle}_ratio_demanda': ratio_demanda_ciclo,
                f'{cycle}_ratio_admision_capacidad': ratio_admision_vs_capacidad,
                f'{cycle}_eficiencia_admision': eficiencia_admision,
                f'{cycle}_activo': 1 if plazas_estimadas > 0 or solicitudes_presentadas > 0 else 0
            })

            # Acumular totales si el ciclo está activo
            if plazas_estimadas > 0 or solicitudes_presentadas > 0:
                ciclos_activos += 1
                total_plazas_estimadas += plazas_estimadas
                total_max_matriculados += max_matriculados
                total_matriculados_año_objetivo += matriculados_año_objetivo
                total_solicitudes_presentadas += solicitudes_presentadas
                total_solicitudes_admitidas += solicitudes_admitidas_corregidas

        # Solo incluir centros que tienen al menos un ciclo activo
        if ciclos_activos > 0:
            # Calcular eficiencia de admisión del centro usando solo solicitudes reales
            total_solicitudes_admitidas_reales = 0
            for cycle in valid_cycles:
                if center_data.get(f'{cycle}_activo', 0) == 1:
                    # Para la eficiencia del centro, usar solo solicitudes admitidas reales (sin transiciones)
                    total_solicitudes_admitidas_reales += center_data[f'{cycle}_solicitudes_admitidas']

            # Métricas agregadas del centro
            center_data.update({
                'ciclos_activos': ciclos_activos,
                'total_plazas_estimadas': total_plazas_estimadas,
                'total_matriculados': total_matriculados_año_objetivo,
                'total_max_matriculados': total_max_matriculados,
                'total_solicitudes_presentadas': total_solicitudes_presentadas,
                'total_solicitudes_admitidas': total_solicitudes_admitidas,
                'total_solicitudes_admitidas_reales': total_solicitudes_admitidas_reales,
                'tasa_ocupacion_centro': (total_matriculados_año_objetivo / total_max_matriculados) if total_max_matriculados > 0 else 0,
                'ratio_demanda_centro': (total_solicitudes_presentadas / total_plazas_estimadas) if total_plazas_estimadas > 0 else 0,
                'ratio_admision_capacidad_centro': (total_solicitudes_admitidas / total_plazas_estimadas) if total_plazas_estimadas > 0 else 0,
                'eficiencia_admision_centro': (total_solicitudes_admitidas_reales / total_solicitudes_presentadas) if total_solicitudes_presentadas > 0 else 0
            })

            center_analysis.append(center_data)

    return pd.DataFrame(center_analysis)

# Ejecutar análisis
centers_df = analyze_center_occupancy(data, cycle_config)
centers_activos = centers_df[centers_df['ciclos_activos'] > 0]  # Solo centros con actividad
print(f"✅ Análisis completado para {len(centers_activos)} centros educativos activos")


In [ ]:
# === ESTADÍSTICAS GENERALES DE OCUPACIÓN ===
print("📊 ESTADÍSTICAS DE OCUPACIÓN - SOLICITUDES VS CAPACIDAD")
print("="*65)

# Estadísticas principales
stats_ocupacion = pd.DataFrame({
    'Métrica': [
        'Centros Educativos Activos',
        'Total Plazas Estimadas',
        'Total Estudiantes Matriculados',
        'Total Estudiantes Max Matriculados',
        'Total Solicitudes Presentadas',
        'Total Solicitudes Admitidas',
        'Tasa Ocupación Promedio (%)',
        'Ratio Demanda/Capacidad Promedio',
        'Ratio Admisión/Capacidad Promedio',
        'Eficiencia Admisión Promedio (%)',
        'Centros Sobredemandados (>1.5x)',
        'Centros Subocupados (<50%)'
    ],
    'Valor': [
        len(centers_activos),
        centers_activos['total_plazas_estimadas'].sum(),
        centers_activos['total_matriculados'].sum(),
        centers_activos['total_max_matriculados'].sum(),
        centers_activos['total_solicitudes_presentadas'].sum(),
        centers_activos['total_solicitudes_admitidas'].sum(),
        centers_activos['tasa_ocupacion_centro'].mean() * 100,
        centers_activos['ratio_demanda_centro'].mean(),
        centers_activos['ratio_admision_capacidad_centro'].mean(),
        centers_activos['eficiencia_admision_centro'].mean() * 100,
        (centers_activos['ratio_demanda_centro'] > 1.5).sum(),
        (centers_activos['tasa_ocupacion_centro'] < 0.5).sum()
    ]
})

display(stats_ocupacion.round(2))


In [ ]:
# === CENTROS CON MAYOR PRESIÓN DE DEMANDA ===
print("\n🔥 CENTROS CON MAYOR PRESIÓN DE DEMANDA (Top 15)")
print("Ratio Demanda/Capacidad más alto")
print("="*70)

top_demanda = centers_activos.nlargest(15, 'ratio_demanda_centro')[
    ['center_name', 'ciclos_activos', 'total_solicitudes_presentadas', 'total_plazas_estimadas',
     'ratio_demanda_centro', 'eficiencia_admision_centro']
].copy()

top_demanda.columns = ['Centro', 'Ciclos', 'Solicitudes', 'Plazas Est.', 'Ratio Demanda', 'Eficiencia %']
top_demanda['Ratio Demanda'] = top_demanda['Ratio Demanda'].round(2)
top_demanda['Eficiencia %'] = (top_demanda['Eficiencia %'] * 100).round(1)

display(top_demanda)


In [ ]:
# === CENTROS CON MAYOR OCUPACIÓN ===
print("\n📈 CENTROS CON MAYOR OCUPACIÓN (Top 15)")
print("Tasa de ocupación más alta")
print("="*70)

top_ocupacion = centers_activos.nlargest(15, 'tasa_ocupacion_centro')[
    ['center_name', 'ciclos_activos', 'total_matriculados', 'total_max_matriculados', 'total_plazas_estimadas',
     'tasa_ocupacion_centro', 'ratio_demanda_centro']
].copy()

top_ocupacion.columns = ['Centro', 'Ciclos', 'Matriculados', 'Total Max Matriculados', 'Plazas Est.', 'Ocupación %', 'Ratio Demanda']
top_ocupacion['Ocupación %'] = (top_ocupacion['Ocupación %'] * 100).round(1)
top_ocupacion['Ratio Demanda'] = top_ocupacion['Ratio Demanda'].round(2)

display(top_ocupacion)


In [ ]:
# === ANÁLISIS POR CICLOS EDUCATIVOS ===
print("\n📚 ANÁLISIS POR CICLOS - OCUPACIÓN Y DEMANDA")
print("="*55)

cycle_summary = []
for cycle, config in cycle_config.items():
    # Centros que ofrecen este ciclo
    cycle_centros = centers_activos[centers_activos[f'{cycle}_activo'] == 1]

    if len(cycle_centros) > 0:
        # Métricas agregadas del ciclo usando columnas corregidas
        total_max_matriculados = cycle_centros[f'{cycle}_max_matriculados'].sum()
        total_matriculados_objetivo = cycle_centros[f'{cycle}_matriculados_objetivo'].sum()
        total_plazas = cycle_centros[f'{cycle}_plazas_estimadas'].sum()
        total_solicitudes = cycle_centros[f'{cycle}_solicitudes_presentadas'].sum()
        total_admitidas = cycle_centros[f'{cycle}_solicitudes_admitidas_corregidas'].sum()

        # Transiciones para primaria
        total_transiciones = 0
        if cycle == 'primaria':
            total_transiciones = cycle_centros[f'{cycle}_transiciones_infantil'].sum()

        # Calcular promedios
        tasa_ocupacion_promedio = cycle_centros[f'{cycle}_tasa_ocupacion'].mean()
        ratio_demanda_promedio = cycle_centros[f'{cycle}_ratio_demanda'].mean()
        ratio_admision_promedio = cycle_centros[f'{cycle}_ratio_admision_capacidad'].mean()
        eficiencia_promedio = cycle_centros[f'{cycle}_eficiencia_admision'].mean()

        cycle_summary.append({
            'Ciclo': config['name'],
            'Centros': len(cycle_centros),
            'Max Matriculados': total_max_matriculados,
            'Matriculados 2022-23': total_matriculados_objetivo,
            'Plazas Est.': total_plazas,
            'Solicitudes': total_solicitudes,
            'Admitidas': total_admitidas,
            'Transiciones': total_transiciones,
            'Ocupación %': tasa_ocupacion_promedio * 100,
            'Ratio Demanda': ratio_demanda_promedio,
            'Ratio Admisión': ratio_admision_promedio,
            'Eficiencia %': eficiencia_promedio * 100
        })

cycle_summary_df = pd.DataFrame(cycle_summary)
display(cycle_summary_df.round(1))


In [ ]:
# === EJEMPLOS DE CÁLCULO DE PLAZAS ESTIMADAS ===
print("\n🔍 EJEMPLOS DE CÁLCULO DE PLAZAS ESTIMADAS (Top 5 por plazas)")
print("Usando máximo de (matriculados + admitidos) ÷ años entre todos los años")
print("="*80)

# Seleccionar los 5 centros con más plazas estimadas para mostrar ejemplos
top_capacity_centers = centers_activos.nlargest(5, 'total_plazas_estimadas')

ejemplos_calculo = []
for _, center in top_capacity_centers.iterrows():
    center_ejemplo = {
        'Centro': center['center_name'],
        'Total Plazas Est.': center['total_plazas_estimadas']
    }

    # Mostrar cálculo para cada ciclo activo
    detalles_ciclos = []
    for cycle, config in cycle_config.items():
        if center[f'{cycle}_activo'] == 1:
            max_mat = center[f'{cycle}_max_matriculados']
            max_adm = center[f'{cycle}_max_admitidos']
            max_total = center[f'{cycle}_max_total_estudiantes']
            año_max = center[f'{cycle}_año_max']
            plazas = center[f'{cycle}_plazas_estimadas']
            años_perm = config['años_permanencia']

            detalle = f"{config['name']}: ({max_mat} mat. + {max_adm} adm.) = {max_total} ({año_max}) ÷ {años_perm} años = {plazas:.1f} plazas"
            detalles_ciclos.append(detalle)

    center_ejemplo['Cálculo Detallado'] = ' | '.join(detalles_ciclos)
    ejemplos_calculo.append(center_ejemplo)

ejemplos_df = pd.DataFrame(ejemplos_calculo)
for _, ejemplo in ejemplos_df.iterrows():
    print(f"\n🏫 {ejemplo['Centro']}")
    print(f"   Total Plazas Estimadas: {ejemplo['Total Plazas Est.']:.1f}")
    print(f"   Cálculo: {ejemplo['Cálculo Detallado']}")


In [ ]:
# === CENTROS CON CORRECCIÓN DE PRIMARIA ===
print("\n🔄 CENTROS CON TRANSICIONES AUTOMÁTICAS INFANTIL II → PRIMARIA")
print("="*65)

centros_con_transiciones = centers_activos[
    centers_activos['primaria_transiciones_infantil'] > 0
].copy()

if len(centros_con_transiciones) > 0:
    transiciones_detalle = centros_con_transiciones[
        ['center_name', 'primaria_matriculados_objetivo', 'primaria_solicitudes_admitidas',
         'primaria_transiciones_infantil', 'primaria_solicitudes_admitidas_corregidas']
    ].copy()

    transiciones_detalle.columns = [
        'Centro', 'Matriculados Primaria', 'Admisiones Solicitudes',
        'Transiciones Infantil II', 'Total Admisiones Corregidas'
    ]

    # Mostrar los primeros 10
    display(transiciones_detalle.head(10).round(1))

    print(f"\n📊 Resumen de transiciones:")
    print(f"   • Centros con transiciones: {len(centros_con_transiciones)}")
    print(f"   • Total transiciones automáticas: {centros_con_transiciones['primaria_transiciones_infantil'].sum():.1f}")
    print(f"   • Promedio transiciones por centro: {centros_con_transiciones['primaria_transiciones_infantil'].mean():.1f}")
else:
    print("No se encontraron centros con transiciones automáticas")


In [ ]:
# === ANÁLISIS DE EFICIENCIA POR CENTRO ===
print("\n⚡ ANÁLISIS DE EFICIENCIA - ADMISIÓN VS CAPACIDAD")
print("="*60)

# Clasificar centros por eficiencia
def clasificar_eficiencia(ratio_admision):
    if ratio_admision >= 1.0:
        return "Sobrecapacidad"
    elif ratio_admision >= 0.8:
        return "Alta eficiencia"
    elif ratio_admision >= 0.5:
        return "Eficiencia media"
    else:
        return "Baja eficiencia"

centers_activos['clasificacion_eficiencia'] = centers_activos['ratio_admision_capacidad_centro'].apply(clasificar_eficiencia)

eficiencia_resumen = centers_activos['clasificacion_eficiencia'].value_counts()
eficiencia_porcentajes = (eficiencia_resumen / len(centers_activos) * 100).round(1)

eficiencia_stats = pd.DataFrame({
    'Clasificación': eficiencia_resumen.index,
    'Centros': eficiencia_resumen.values,
    'Porcentaje': eficiencia_porcentajes.values
})

print("Distribución de eficiencia de centros:")
display(eficiencia_stats)

print("\n✅ ANÁLISIS DE OCUPACIÓN EDUCATIVA COMPLETADO")
print("="*60)
